In [2]:
QUESTS = [
    {"Name": "Orange Quest",
     "Type": "Warfare",
     "Requirements": {"orange": 2},
     "Rewards": {"VP": 4}},
    {"Name": "Black Quest",
     "Type": "Skullduggery",
     "Requirements": {"black": 2},
     "Rewards": {"VP": 4}},
]

In [3]:
from collections import namedtuple
Quest = namedtuple("Quest", "name type requirements rewards")

QUESTS2 = [
    Quest("Orange Quest", "Warfare", {"orange": 2}, {"VP": 4}),
    Quest("Black Quest", "Skullduggery", {"orange": 2}, {"VP": 4})
]

In [4]:
print(QUESTS)

[{'Name': 'Orange Quest', 'Type': 'Warfare', 'Requirements': {'orange': 2}, 'Rewards': {'VP': 4}}, {'Name': 'Black Quest', 'Type': 'Skullduggery', 'Requirements': {'black': 2}, 'Rewards': {'VP': 4}}]


In [5]:
print(QUESTS2)

[Quest(name='Orange Quest', type='Warfare', requirements={'orange': 2}, rewards={'VP': 4}), Quest(name='Black Quest', type='Skullduggery', requirements={'orange': 2}, rewards={'VP': 4})]
